# Code for extracting datasets for each day from each of the multiomics datasets

The jupyter notebook was created by Joey Reyna, LJI:

Source: https://github.com/joreynajr/cmi-pb-multiomics/blob/main/workflow/notebooks/development/Standardize_data.ipynb

The code has been slightly modifided to obtain both TMP and RAW gene expression data.

In [5]:
import os 
import pandas as pd 
import numpy as np
# Change to the current directory
os.chdir('/Users/mrasmussen/OneDrive - Danmarks Tekniske Universitet/Teknisk_Biomedicin\
/6_semester/Bachelorproject/Data/Pre_process/cmi-pb-multiomics')
# Change to ouput directory
outdir = 'results/main/cmi_pb_datasets/processed/'
os.makedirs(outdir, exist_ok=True)

In [6]:
assays = ('cytof', 'olink', 'rnaseq', 'abtiters')
longnames = ('live_cell_percentages', 'olink_prot_exp', 'rnaseq', 'ab_titer')

In [7]:
# loading gencode data
gencode = pd.read_table('results/refs/gencode/gencode.v38lift37.annotation.protein_coding.bed', header=None)
gencode.columns = ['chr', 'start', 'end', 'strand', 'gene_id', 'gene_name']
gencode['gene_nonversioned_id'] = gencode['gene_id'].str.replace('\..*', '', regex=True)

In [8]:
interm_dir = os.path.join(outdir, 'full')

## Load the 2020 data

In [9]:
twenty_data = {}

# get meta master table
for metatable in ('subject', 'specimen'): 
    fn = 'results/main/cmi_pb_datasets/raw/2020LD_{}.csv'.format(metatable)
    df = pd.read_table(fn, sep=',')
    twenty_data[metatable] = df
    
subjects = twenty_data['subject']
specimen = twenty_data['specimen']
master_meta = subjects.merge(specimen, on='subject_id')
master_meta = master_meta[['subject_id',
                           'specimen_id',
                           'infancy_vac',
                           'biological_sex',
                           'year_of_birth',
                           'date_of_boost',
                           'actual_day_relative_to_boost',
                           'planned_day_relative_to_boost',
                           'ethnicity',
                           'race',
                           'dataset',
                           'specimen_type',
                           'visit']]

# defining the meta columns (used to remove columns later one)
meta_cols = ['specimen_id', 'infancy_vac', 'biological_sex',
             'year_of_birth', 'date_of_boost', 'actual_day_relative_to_boost',
             'ethnicity', 'race', 'dataset',
             'specimen_type', 'visit']

twenty_data['master_meta'] = master_meta

# save meta
meta_fn = os.path.join(interm_dir, 'meta.2020.pivoted.tsv.gz')
twenty_data['master_meta'].to_csv(meta_fn, sep='\t')

In [10]:
# code to switch between different MFI's 
curr_mfi, curr_mfi_name = ('MFI', 'mfi_raw')
#curr_mfi, curr_mfi_name = ('MFI_normalised', 'mfi_normalised')

In [7]:
for i, longname in enumerate(longnames): 
    fn = 'results/main/cmi_pb_datasets/raw/2020LD_{}.csv'.format(longname)
    df = pd.read_table(fn, sep=',')
    
    print(fn)
    
    if assays[i] == 'rnaseq':
        df.loc[:, 'ensembl_gene_id'] = df.loc[:, 'versioned_ensembl_gene_id'].str.replace('\.[0-9]+', '', regex=True)
        df_tpm = df.pivot(index='specimen_id', columns='ensembl_gene_id', values='tpm')
        df_tpm = master_meta.merge(df_tpm, on='specimen_id')
        df_raw = df.pivot(index='specimen_id', columns='ensembl_gene_id', values='raw_count')
        df_raw = master_meta.merge(df_raw, on='specimen_id')
        
    elif assays[i] == 'cytof':
        df = df.pivot(index='specimen_id', columns='cell_type_name', values='percent_live_cell')
        df = master_meta.merge(df, on='specimen_id')
        
    elif assays[i] == 'olink':
        # Still getting duplicate specimens? 
        df = df[df['unit'] == "Normalized Protein eXpression"]
        df = df[df['quality_control'] == 'Pass']
        df = df[df['protein_expression'] > df['lower_limit_of_quantitation']]
        df = df.pivot(index='specimen_id', columns='olink_id', values='protein_expression')
        df = master_meta.merge(df, on='specimen_id')      
        
    else:
        aglist = ['1% PFA PT', 'PT', 'PRN', 'FHA', 'FIM2/3']
        df = df.loc[df.antigen.isin(aglist), :]
        df['antigen'].replace(to_replace='1% PFA PT', value='PT', inplace=True)
        df['isotype_antigen'] = df['isotype'] + '-' + df['antigen']
        df = df.pivot(index='specimen_id', columns='isotype_antigen', values='ab_titer', values=curr_mfi)
        df = master_meta.merge(df, on='specimen_id')  
    
    if assays[i] == 'rnaseq':
        rna_tpm = assays[i] + '_tpm'
        twenty_data[rna_tpm] = df_tpm.drop(meta_cols, axis=1)
        rna_raw = assays[i] + '_raw'
        twenty_data[rna_raw] = df_raw.drop(meta_cols, axis=1)
    else:
        twenty_data[assays[i]] = df.drop(meta_cols, axis=1)
    

results/main/cmi_pb_datasets/raw/2020LD_live_cell_percentages.csv
results/main/cmi_pb_datasets/raw/2020LD_olink_prot_exp.csv
results/main/cmi_pb_datasets/raw/2020LD_rnaseq.csv
results/main/cmi_pb_datasets/raw/2020LD_ab_titer.csv


In [8]:
for assay, df in twenty_data.items():
    print(assay, df.subject_id.nunique())

subject 60
specimen 60
master_meta 60
cytof 20
olink 18
rnaseq_tpm 36
rnaseq_raw 36
abtiters 58


In [12]:
# save a dataframe for each assay on each day
for assay, df in twenty_data.items():
    if assay in ['cytof', 'olink', 'rnaseq_tpm', 'rnaseq_raw', 'abtiters']:
        print(assay)
        for day, day_df in df.groupby('planned_day_relative_to_boost'):
            
            if assay == 'abtiters': 
                outfn = os.path.join(interm_dir, '{}.{}.2020.day{}.pivoted.tsv'.format(assay, curr_mfi_name, day))
            else:
                outfn = os.path.join(interm_dir, '{}.2020.day{}.pivoted.tsv'.format(assay, day))
                
            day_df.drop('planned_day_relative_to_boost', axis=1).to_csv(outfn, index=False, sep='\t')
            print(day, day_df.shape)

cytof
0 (20, 27)
1 (20, 27)
3 (20, 27)
7 (20, 27)
14 (20, 27)
olink
0 (18, 256)
1 (18, 256)
3 (18, 256)
7 (18, 256)
14 (18, 256)
rnaseq_tpm
0 (36, 58349)
1 (36, 58349)
3 (36, 58349)
7 (36, 58349)
14 (36, 58349)
rnaseq_raw
0 (36, 58349)
1 (36, 58349)
3 (36, 58349)
7 (36, 58349)
14 (36, 58349)
abtiters
0 (58, 25)
1 (57, 25)
3 (57, 25)
7 (57, 25)
14 (57, 25)
30 (54, 25)
90 (51, 25)
386 (1, 25)
402 (1, 25)
428 (1, 25)


# Load the 2021 data

In [14]:
twentyone_data = {}

# get meta master table
for metatable in ('subject', 'specimen'): 
    fn = 'results/main/cmi_pb_datasets/raw/2021BD_{}.csv'.format(metatable)
    df = pd.read_table(fn, sep=',')
    twentyone_data[metatable] = df
    
subjects = twentyone_data['subject']
specimen = twentyone_data['specimen']
master_meta = subjects.merge(specimen, on='subject_id')
master_meta = master_meta[['subject_id',
                           'specimen_id',
                           'infancy_vac',
                           'biological_sex',
                           'year_of_birth',
                           'date_of_boost',
                           'actual_day_relative_to_boost',
                           'planned_day_relative_to_boost',
                           'ethnicity',
                           'race',
                           'dataset',
                           'specimen_type',
                           'visit']]
twentyone_data['master_meta'] = master_meta

In [15]:
for i, longname in enumerate(longnames): 
        
    fn = 'results/main/cmi_pb_datasets/raw/2021BD_{}.csv'.format(longname)
    df = pd.read_table(fn, sep=',')
    
    if assays[i] == 'rnaseq':
        df.loc[:, 'ensembl_gene_id'] = df.loc[:, 'versioned_ensembl_gene_id'].str.replace('\.[0-9]+', '', regex=True)
        df_tpm = df.pivot(index='specimen_id', columns='ensembl_gene_id', values='tpm')
        df_tpm = master_meta.merge(df_tpm, on='specimen_id')
        df_raw = df.pivot(index='specimen_id', columns='ensembl_gene_id', values='raw_count')
        df_raw = master_meta.merge(df_raw, on='specimen_id')
        
        df.loc[:, 'ensembl_gene_id'] = df.loc[:, 'versioned_ensembl_gene_id'].str.replace('\.[0-9]+', '', regex=True)
        df = df.pivot(index='specimen_id', columns='ensembl_gene_id', values='raw_count')
        df = master_meta.merge(df, on='specimen_id')
                
    elif assays[i] == 'cytof':
        df = df.pivot(index='specimen_id', columns='cell_type_name', values='percent_live_cell')
        df = master_meta.merge(df, on='specimen_id')
        
    elif assays[i] == 'olink':
        df = df.pivot(index='specimen_id', columns='olink_id', values='protein_expression')
        df = master_meta.merge(df, on='specimen_id')       
        
    else:
        aglist = ['PT', 'PRN', 'FHA', 'FIM2/3']
        df = df.loc[df.antigen.isin(aglist), :]
        df['isotype_antigen'] = df['isotype'] + '-' + df['antigen']
        df = df.pivot(index='specimen_id', columns='isotype_antigen', values='ab_titer', values=curr_mfi)
        df = master_meta.merge(df, on='specimen_id')  
        
    if assays[i] == 'rnaseq':
        rna_tpm = assays[i] + '_tpm'
        twentyone_data[rna_tpm] = df_tpm.drop(meta_cols, axis=1)
        rna_raw = assays[i] + '_raw'
        twentyone_data[rna_raw] = df_raw.drop(meta_cols, axis=1)
    else:
        twentyone_data[assays[i]] = df.drop(meta_cols, axis=1)
    

In [16]:
longname = 'live_cell_percentages'
df = 'results/main/cmi_pb_datasets/raw/2021BD_{}.csv'.format(longname)

df = pd.read_table(df, sep=',')

df = df.pivot(index='specimen_id', columns='cell_type_name', values='percent_live_cell')
df = master_meta.merge(df, on='specimen_id')


In [17]:
for assay, df in twentyone_data.items():
    print(assay, df.subject_id.nunique())

subject 36
specimen 36
master_meta 36
cytof 33
olink 36
rnaseq_tpm 36
rnaseq_raw 36
abtiters 33


In [19]:
os.makedirs(interm_dir, exist_ok=True)

# save abtiters  
abtiters_fn = os.path.join(interm_dir, 'abtiters.2021.day0.pivoted.tsv.gz'.format(curr_mfi_name))
twentyone_data['abtiters'].drop('planned_day_relative_to_boost', axis=1).to_csv(abtiters_fn, index=False, sep='\t')

# save cytof 
cytof_fn = os.path.join(interm_dir, 'cytof.2021.day0.pivoted.tsv.gz')
twentyone_data['cytof'].drop('planned_day_relative_to_boost', axis=1).to_csv(cytof_fn, index=False, sep='\t')

# save olink
olink_fn = os.path.join(interm_dir, 'olink.2021.day0.pivoted.tsv.gz')
twentyone_data['olink'].drop('planned_day_relative_to_boost', axis=1).to_csv(olink_fn, index=False, sep='\t')

# save rnaseq_tpm
rnaseq_fn = os.path.join(interm_dir, 'rnaseq_tpm.2021.day0.pivoted.tsv.gz')
twentyone_data['rnaseq_tpm'].drop('planned_day_relative_to_boost', axis=1).to_csv(rnaseq_fn, index=False, sep='\t')

# save rnaseq_raw
rnaseq_fn = os.path.join(interm_dir, 'rnaseq_raw.2021.day0.pivoted.tsv.gz')
twentyone_data['rnaseq_raw'].drop('planned_day_relative_to_boost', axis=1).to_csv(rnaseq_fn, index=False, sep='\t')

# save meta
meta_fn = os.path.join(interm_dir, 'meta.2021.pivoted.tsv.gz')
twentyone_data['master_meta'].to_csv(meta_fn, index=False, sep='\t')

In [23]:
# summary21 = []
# for assay, df in twentyone_data.items():
#     summary21.append([assay, df.subject_id.nunique()])

# summary20 = []
# for assay, df in twenty_data.items():
#     summary20.append([df.subject_id.nunique()])

# summary = pd.concat([pd.DataFrame(summary21), pd.DataFrame(summary20)], axis=1)
# summary.columns = ('table', 'nsamples21', 'nsamples20')
# summary = summary[['table', 'nsamples20', 'nsamples21']]

In [24]:
# fn = 'results/main/cmi_pb_datasets/2020LD_{}.csv'.format('olink_prot_exp')
# df = pd.read_table(fn, sep=',')

## Standardize RNA-seq

The RNA-seq dataset includes a lot more variables and processing so there are many steps we are including for standardization:

    1) remove zero variance genes,
    2) remove mitochondrial genes, 
    3) keep expressed genes (defined as those with TPM > 1) in more than a certain proportion of people (cut_filter)
    4) intersect genes across 2020 and 2021 and finally make harmonized 2020 and 2021 tables 

In [20]:
# load gene data to find mitochondrial genes
mito_genes = gencode.loc[(gencode.chr == 'chrM'), 'gene_nonversioned_id']
mito_genes = set(mito_genes.unique().tolist())

In [21]:
# value used across both RNA-seq datasets
cut_filter = 0.3

### RNA-seq for 2020

In [25]:
# find zero variance genes 
tmp = twenty_data['rnaseq_tpm'].loc[twenty_data['rnaseq_tpm'].planned_day_relative_to_boost == 0]
tmp = tmp.loc[:, tmp.columns.str.match('ENSG')]
tmpvars = tmp.var()
zero_var_genes = tmpvars[(tmpvars == 0)]
zero_var_genes = set(zero_var_genes.index.tolist())

In [26]:
# locate genes which are expressed
# defined as genes with TPM > 1 in more than 30% of samples 
expressed_genes = (tmp > 1).sum() > cut_filter * tmp.shape[0]
expressed_genes = set(expressed_genes.index[expressed_genes.values.tolist()])

In [27]:
# get the final list of genes 
keep_genes = expressed_genes.difference(mito_genes).difference(zero_var_genes)
keep_genes = keep_genes.intersection(gencode.gene_nonversioned_id.values.tolist())

In [28]:
# make the final new_rnaseq table
new_rnaseq1 = tmp.loc[:, ~tmp.columns.str.match('ENSG')]
new_rnaseq2 = tmp.loc[:, keep_genes]
twenty_data['rnaseq_std'] = pd.concat([new_rnaseq1, new_rnaseq2], axis=1)

/var/folders/j7/nys0_gj926n5tgcrhlqh1p7h4yqv41/T/ipykernel_10057/2441285580.py:3: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  new_rnaseq2 = tmp.loc[:, keep_genes]


### RNA-seq for 2021

In [30]:
# find zero variance genes 
tmp = twentyone_data['rnaseq_tpm'].loc[twentyone_data['rnaseq_tpm'].planned_day_relative_to_boost == 0]
tmp = tmp.loc[:, tmp.columns.str.match('ENSG')]
tmpvars = tmp.var()
zero_var_genes = tmpvars[(tmpvars == 0)]
zero_var_genes = set(zero_var_genes.index.tolist())

In [31]:
# locate genes which are expressed
# defined as genes with TPM > 1 in more than 30% of samples 
expressed_genes = (tmp > 1).sum() > cut_filter * tmp.shape[0]
expressed_genes = set(expressed_genes.index[expressed_genes.values.tolist()])

In [32]:
# get the final list of genes 
keep_genes = expressed_genes.difference(mito_genes).difference(zero_var_genes)
keep_genes = keep_genes.intersection(gencode.gene_nonversioned_id.values.tolist())

In [33]:
# make the final new_rnaseq table
new_rnaseq1 = tmp.loc[:, ~tmp.columns.str.match('ENSG')]
new_rnaseq2 = tmp.loc[:, keep_genes]
twentyone_data['rnaseq_std'] = pd.concat([new_rnaseq1, new_rnaseq2], axis=1)

/var/folders/j7/nys0_gj926n5tgcrhlqh1p7h4yqv41/T/ipykernel_10057/2147526914.py:3: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  new_rnaseq2 = tmp.loc[:, keep_genes]


### Harmonize the RNA-seq datasets 

In [34]:
# get the shared genes between 2020 and 2021
twenty_genes = set(twenty_data['rnaseq_std'].columns.str.extract('(ENSG[0-9]+)').dropna()[0])
twentyone_genes = set(twentyone_data['rnaseq_std'].columns.str.extract('(ENSG[0-9]+)').dropna()[0])
shared_genes = sorted(twenty_genes.intersection(twentyone_genes))

In [36]:
# getting harmonized for 2020 
new_rnaseq1 = twenty_data['rnaseq_tpm'].loc[:, ~twenty_data['rnaseq_tpm'].columns.str.match('ENSG')]
new_rnaseq2 = twenty_data['rnaseq_tpm'].loc[:, shared_genes]
harmonized_df = pd.concat([new_rnaseq1, new_rnaseq2], axis=1)
harmonized_df = harmonized_df.loc[harmonized_df.planned_day_relative_to_boost == 0]
harmonized_df.drop('planned_day_relative_to_boost', axis=1, inplace=True)
twenty_data['rnaseq_harmonized'] = harmonized_df

In [37]:
# getting harmonized for 2021
new_rnaseq1 = twentyone_data['rnaseq_tpm'].loc[:, ~twentyone_data['rnaseq_tpm'].columns.str.match('ENSG')]
new_rnaseq2 = twentyone_data['rnaseq_tpm'].loc[:, shared_genes]
harmonized_df = pd.concat([new_rnaseq1, new_rnaseq2], axis=1)
harmonized_df = harmonized_df.loc[harmonized_df.planned_day_relative_to_boost == 0]
harmonized_df.drop('planned_day_relative_to_boost', axis=1, inplace=True)
twentyone_data['rnaseq_harmonized'] = harmonized_df

In [38]:
# save harmonize rnaseq
harmony_dir = os.path.join(outdir, 'harmonized/')
os.makedirs(harmony_dir, exist_ok=True)

rnaseq_2020_fn = os.path.join(harmony_dir, 'rnaseq.2020.day0.pivoted.tsv.gz')
twenty_data['rnaseq_harmonized'].loc[:, ['subject_id'] + shared_genes].to_csv(rnaseq_2020_fn, index=False, sep='\t')

rnaseq_2021_fn = os.path.join(harmony_dir, 'rnaseq.2021.day0.pivoted.tsv.gz')
twentyone_data['rnaseq_harmonized'].loc[:, ['subject_id'] + shared_genes].to_csv(rnaseq_2021_fn, index=False, sep='\t')

In [39]:
twentyone_data['rnaseq_harmonized'].shape

(36, 11589)

## Standardize CyTOF

In [31]:
shared_cells = set(twenty_data['cytof'].columns).intersection(set(twentyone_data['cytof'].columns))
shared_cells = sorted(list(shared_cells))
shared_cells.remove('subject_id')
shared_cells.remove('planned_day_relative_to_boost')

In [32]:
day0_rows = twenty_data['cytof']['planned_day_relative_to_boost'] == 0
twenty_data['cytof_harmonized'] = twenty_data['cytof'].loc[day0_rows]
twenty_data['cytof_harmonized'] = twenty_data['cytof_harmonized'].loc[:, ['subject_id'] + shared_cells]

day0_rows = twentyone_data['cytof']['planned_day_relative_to_boost'] == 0
twentyone_data['cytof_harmonized'] = twentyone_data['cytof'].loc[day0_rows]
twentyone_data['cytof_harmonized'] = twentyone_data['cytof_harmonized'].loc[:, ['subject_id'] + shared_cells]

In [33]:
cytof_2020_fn = os.path.join(harmony_dir, 'cytof.2020.day0.pivoted.tsv.gz')
twenty_data['cytof_harmonized'].to_csv(cytof_2020_fn, index=False, sep='\t')

In [34]:
cytof_2021_fn = os.path.join(harmony_dir, 'cytof.2021.day0.pivoted.tsv.gz')
twentyone_data['cytof_harmonized'].to_csv(cytof_2021_fn, index=False, sep='\t')

In [35]:
twentyone_data['cytof_harmonized'].shape

(33, 23)

## Standardize Ab Titers

In [36]:
abtiters_overlap = set(twenty_data['abtiters'].columns).intersection(twentyone_data['abtiters'].columns.tolist())
abtiters_overlap.discard('planned_day_relative_to_boost')
abtiters_overlap.discard('subject_id')
abtiters_overlap = sorted(abtiters_overlap)

In [37]:
twenty_data['abtiters_harmonized'] = twenty_data['abtiters'].\
        loc[twenty_data['abtiters'].planned_day_relative_to_boost == 0, ]
twenty_data['abtiters_harmonized'] = twenty_data['abtiters_harmonized'].loc[:, ['subject_id'] + abtiters_overlap]
new_abtiters_fn = os.path.join(harmony_dir, 'abtiters.2020.day0.pivoted.tsv.gz'.format(curr_mfi_name))
twenty_data['abtiters_harmonized'].to_csv(new_abtiters_fn, sep='\t', index=False)

In [38]:
twentyone_data['abtiters_harmonized'] = twentyone_data['abtiters'].\
        loc[twentyone_data['abtiters'].planned_day_relative_to_boost == 0, ]
twentyone_data['abtiters_harmonized'] = twentyone_data['abtiters_harmonized'].loc[:, ['subject_id'] + abtiters_overlap]
new_abtiters_fn = os.path.join(harmony_dir, 'abtiters.2021.day0.pivoted.tsv.gz'.format(curr_mfi_name))
twentyone_data['abtiters_harmonized'].to_csv(new_abtiters_fn, sep='\t', index=False)

## Standardize Olink

### Checking Olink features 

In [39]:
def load_api_data(url):
    """
    Loading data using the API.
    """

    import urllib.request, json, ssl 

    gcontext = ssl.SSLContext()  
    with urllib.request.urlopen(url, context=gcontext) as url:
        #data = url.read()
        data = pd.read_json(url)
        return(data)

In [40]:
set(twenty_data['olink'].columns.tolist()).intersection(twentyone_data['olink'].columns.tolist())

{'planned_day_relative_to_boost', 'subject_id'}

In [41]:
# load olink mapper
url = 'https://www.cmi-pb.org:443/api/v2/olink_prot_info'
olink_meta = load_api_data(url) 
olink_meta.set_index('olink_id', inplace=True)

# keep non-duplicated values only
olink_uniq = olink_meta.loc[~olink_meta.index.duplicated(keep=False), ]
olink_uniq = olink_uniq.squeeze().to_dict()

# keep only duplicated values
olink_dups = olink_meta.loc[olink_meta.index.duplicated(keep=False), ]
#olink_uniq = olink_uniq.squeeze().to_dict()

### Checking Olink features 

In [42]:
check_olink_2020 = twenty_data['olink'].copy()
check_olink_2020.rename(columns=olink_uniq, inplace=True)

In [43]:
check_olink_2021 = twentyone_data['olink'].copy()
check_olink_2021.rename(columns=olink_uniq, inplace=True)

In [44]:
olink_overlaps = set(check_olink_2020.columns.tolist()).intersection(check_olink_2021.columns.tolist())
olink_overlaps.discard('planned_day_relative_to_boost')
olink_overlaps.discard('subject_id')
olink_overlaps = sorted(olink_overlaps)

In [45]:
print('There are {} overlaps with the unique Uniprot proteins.'.format(len(olink_overlaps)))

There are 27 overlaps with the unique Uniprot proteins.


In [46]:
dup_overlaps = set(olink_overlaps).intersection(olink_dups.uniprot_id.tolist())
print('There are {} overlaps with the duplicates.'.format(len(dup_overlaps)))

There are 0 overlaps with the duplicates.


In [47]:
s = 'Olink data from 2020 has {} protein measurements'.format(twenty_data['olink'].shape[1] - 2)
s += ' whereas 2021 has {}.'.format(twentyone_data['olink'].shape[1] - 2)
print(s)

Olink data from 2020 has 254 protein measurements whereas 2021 has 45.


### Harmonizing features

In [48]:
twenty_cytof_harmonized = check_olink_2020.copy()
twenty_cytof_harmonized = twenty_cytof_harmonized.loc[twenty_cytof_harmonized.planned_day_relative_to_boost == 0]
twenty_cytof_harmonized = twenty_cytof_harmonized[['subject_id'] + olink_overlaps]
twenty_harmonized_cytof_fn = os.path.join(harmony_dir, 'olink.2020.day0.pivoted.tsv.gz')
twenty_cytof_harmonized.to_csv(twenty_harmonized_cytof_fn, sep='\t', index=False)

In [49]:
twentyone_cytof_harmonized = check_olink_2021.copy()
twentyone_cytof_harmonized = twentyone_cytof_harmonized.loc[twentyone_cytof_harmonized.planned_day_relative_to_boost == 0]
twentyone_cytof_harmonized = twentyone_cytof_harmonized[['subject_id'] + olink_overlaps]
twentyone_harmonized_cytof_fn = os.path.join(harmony_dir, 'olink.2021.day0.pivoted.tsv.gz')
twentyone_cytof_harmonized.to_csv(twentyone_harmonized_cytof_fn, sep='\t', index=False)

## Providing all task vectors for training

In [50]:
from IPython.display import display

In [51]:
# Loading the table describing each task 
tasks = pd.read_table('results/refs/tasks.tsv')

# Making a task vector using GENE IDS for RNA based tasks
i = 1
for idd, task in tasks.iterrows():
    
    # extracting the required columns
    cols = ['subject_id', task.fetchname]
    task_data = twenty_data[task.assay]
    task_data = task_data.loc[(task_data.planned_day_relative_to_boost == task.day), cols]
    task_data.columns = ['subject_id', task.fetchname + '_day' + str(task.day)]    
    
    # merging the data together
    if i == 1: 
        master_tasks = task_data
    else:
        master_tasks = master_tasks.merge(task_data, on='subject_id', how='outer')
    i += 1 

# saving the task matrix 
task_fn = os.path.join(harmony_dir, 'task_matrix.feature_names.tsv.gz'.format(curr_mfi_name))
master_tasks.to_csv(task_fn, sep='\t', index=False)

# Making a task vector using GENE NAMES for RNA based tasks
i = 1
for idd, task in tasks.iterrows():
    
    # extracting the required columns
    cols = ['subject_id', task.fetchname]
    task_data = twenty_data[task.assay]
    task_data = task_data.loc[(task_data.planned_day_relative_to_boost == task.day), cols]
    
    if task.assay == 'rnaseq':
        task_data.columns = ['subject_id', task.fullname + '_day' + str(task.day)]    
    else:
        task_data.columns = ['subject_id', task.fetchname + '_day' + str(task.day)]    
    
    # merging the data together
    if i == 1: 
        master_tasks = task_data
    else:
        master_tasks = master_tasks.merge(task_data, on='subject_id', how='outer')
    i += 1 

# saving the task matrix 
task_fn = os.path.join(harmony_dir, 'task_matrix.common_names.tsv.gz'.format(curr_mfi_name))
master_tasks.to_csv(task_fn, sep='\t', index=False)

In [ ]:
twenty_data.keys()

In [52]:
master_tasks.head()

,subject_id,IgG-PT_day14,IgG-FHA_day14,IgG-PRN_day14,Monocytes_day1,ASCs (Plasmablasts)_day7,CD4Tcells_day3,CCL3_day3,IL-6_day3,NFKBIA_day7,XIST_day14
0,1,229.603692,0.064000,1039.814648,NaN,NaN,NaN,46.410,3.814,651.685,146.247
1,3,129.197956,640.894817,1030.816658,NaN,NaN,NaN,26.204,2.646,575.733,121.084
2,4,105.426574,162.807830,874.712337,7.211965,1.77,8.700555,13.353,3.770,550.870,0.132
3,5,97.743258,0.064000,646.092906,NaN,NaN,NaN,20.618,2.070,1098.404,0.066
4,6,162.500000,137.831579,58.431482,41.380502,3.50,41.706336,19.606,2.163,686.300,127.023


## Providing day 0 data for all task vectors (for fold change transformations)|

In [ ]:
# Making a task vector using GENE IDS for RNA based tasks
i = 1
for idd, task in tasks.iterrows():
    
    # extracting the required columns
    cols = ['subject_id', task.fetchname]
    task_data = twenty_data[task.assay]
    task_data = task_data.loc[(task_data.planned_day_relative_to_boost == 0), cols]
    task_data.columns = ['subject_id', task.fetchname + '_day' + str(0)]    
    
    # merging the data together
    if i == 1: 
        zero_day_tasks = task_data
    else:
        zero_day_tasks = zero_day_tasks.merge(task_data, on='subject_id', how='outer')
    i += 1 

# saving the task matrix 
task_fn = os.path.join(harmony_dir, 'zero_day_matrix.feature_names.{}.tsv'.format(curr_mfi_name))
zero_day_tasks.to_csv(task_fn, sep='\t', index=False)

In [ ]:
# Making a task vector using GENE NAMES for RNA based tasks
i = 1
for idd, task in tasks.iterrows():
    
    # extracting the required columns
    cols = ['subject_id', task.fetchname]
    task_data = twenty_data[task.assay]
    task_data = task_data.loc[(task_data.planned_day_relative_to_boost == 0), cols]
    
    if task.assay == 'rnaseq':
        task_data.columns = ['subject_id', task.fullname + '_day' + str(0)]    
    else:
        task_data.columns = ['subject_id', task.fetchname + '_day' + str(0)]    
    
    # merging the data together
    if i == 1: 
        zero_day_tasks = task_data
    else:
        zero_day_tasks = zero_day_tasks.merge(task_data, on='subject_id', how='outer')
    i += 1 

# saving the task matrix 
task_fn = os.path.join(harmony_dir, 'zero_day_matrix.common_names.{}.tsv'.format(curr_mfi_name))
zero_day_tasks.to_csv(task_fn, sep='\t', index=False)

In [ ]:
head(zero_day_tasks)

## Providing meta data into a basic format

In [53]:
twenty_basic_meta = twenty_data['master_meta'].drop_duplicates('subject_id')
drop_cols = ['specimen_id', 'actual_day_relative_to_boost', 'planned_day_relative_to_boost', 'visit', 'dataset']
twenty_basic_meta.drop(drop_cols, axis=1, inplace=True)

twenty_basic_fn = os.path.join(harmony_dir, 'clinical_metadata.2020.tsv.gz')
twenty_basic_meta.to_csv(twenty_basic_fn, sep='\t', index=False)

/mnt/BioHome/jreyna/.local/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [54]:
twentyone_basic_meta = twentyone_data['master_meta'].drop_duplicates('subject_id')
twentyone_basic_meta.drop(drop_cols, axis=1, inplace=True)

twentyone_basic_fn = os.path.join(harmony_dir, 'clinical_metadata.2021.tsv.gz')
twentyone_basic_meta.to_csv(twentyone_basic_fn, sep='\t', index=False)

## Summarize the harmonization process

In [ ]:
summary = []
for assay in assays:
    
    harmonized_assay = '{}_harmonized'.format(assay)
    summary.append([assay, 
                    twenty_data[assay].shape[1],
                    twentyone_data[assay].shape[1], 
                    twentyone_data[harmonized_assay].shape[1]])

summary = pd.DataFrame(summary)
summary.columns = ['Assay', '2020', '2021', 'Harmonized']


In [ ]:
summary.sort_values('Assay')